# Network with Applications

In this file, we'll demonstrate the simulation of a more complicated network topology with randomized applications. The network topology, including hardware components, is shown below:

<img src="./notebook_images/star_network.png" width="700"/>

In [3]:
from numpy import random
import time

In [4]:
from sequence.topology.topology import Topology
from sequence.kernel.timeline import Timeline
from sequence.app.random_request import RandomRequestApp

In [5]:
def set_parameters(topology):
    # set memory parameters
    MEMO_FREQ = 2e3
    MEMO_EXPIRE = 1.1
    MEMO_EFFICIENCY = 1
    MEMO_FIDELITY = 0.9349367588934053
    for node in topology.get_nodes_by_type("QuantumRouter"):
        node.memory_array.update_memory_params("frequency", MEMO_FREQ)
        node.memory_array.update_memory_params("coherence_time", MEMO_EXPIRE)
        node.memory_array.update_memory_params("efficiency", MEMO_EFFICIENCY)
        node.memory_array.update_memory_params("raw_fidelity", MEMO_FIDELITY)

    # set entanglement swapping parameters
    SWAP_SUCC_PROB = 0.64
    SWAP_DEGRADATION = 0.99
    for node in topology.get_nodes_by_type("QuantumRouter"):
        node.network_manager.protocol_stack[1].set_swapping_success_rate(SWAP_SUCC_PROB)
        node.network_manager.protocol_stack[1].set_swapping_degradation(SWAP_DEGRADATION)
        
    # set detector parameters
    DETECTOR_EFFICIENCY = 0.8
    DETECTOR_COUNT_RATE = 5e7
    DETECTOR_RESOLUTION = 100
    for node in topology.get_nodes_by_type("BSMNode"):
        node.bsm.update_detectors_params("efficiency", DETECTOR_EFFICIENCY)
        node.bsm.update_detectors_params("count_rate", DETECTOR_COUNT_RATE)
        node.bsm.update_detectors_params("time_resolution", DETECTOR_RESOLUTION)
        
    # set quantum channel parameters
    ATTENUATION = 0.0002
    QC_FREQ = 1e11
    for qc in topology.qchannels:
        qc.attenuation = ATTENUATION
        qc.frequency = QC_FREQ

In [6]:
def test(sim_time):
    random.seed(0)
    network_config = "star_network.json"
    
    tl = Timeline(sim_time * 1e9)
    network_topo = Topology("network_topo", tl)
    network_topo.load_config(network_config)
    
    set_parameters(network_topo)
    
    # construct random request applications
    node_names = [node.name for node in network_topo.get_nodes_by_type("QuantumRouter")]
    apps = []
    for i, name in enumerate(node_names):
        other_nodes = node_names[:] # copy node name list
        other_nodes.remove(name)
        app = RandomRequestApp(network_topo.nodes[name], other_nodes, i)
        apps.append(app)
        app.start()
        
    tl.init()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    for app in apps:
        print(app.node.name)
        print("\t", len(app.get_wait_time()))
        print("\t", app.get_wait_time())
        throughput = app.get_throughput()
        print("\t", app.reserves)
        print("\t", throughput)

In [8]:
test(50e3)

execution time 1119.42 sec
center
	 3
	 [1600000000000.0, 3700000000000.0, 2300000000000.0]
	 [['edge4', 1600000000000.0, 16600000000000.0, 14, 0.808194704787239], ['edge1', 20300000000001.0, 36300000000001.0, 18, 0.9825511154555444], ['edge3', 38600000000002.0, 56600000000002.0, 18, 0.8005477000340296]]
	 [1756.6, 74.6875]
edge1
	 1
	 [42241000000000.0]
	 [['edge2', 42241000000000.0, 59241000000000.0, 16, 0.8288319225439268]]
	 []
edge2
	 1
	 [1200000000000.0]
	 [['edge4', 1200000000000.0, 12200000000000.0, 14, 0.962845148118856]]
	 [27.363636363636363]
edge3
	 0
	 []
	 []
	 []
edge4
	 2
	 [12508000000000.0, 1300000000000.0]
	 [['edge2', 12508000000000.0, 23508000000000.0, 16, 0.9952487411415408], ['edge2', 24808000000001.0, 41808000000001.0, 16, 0.9331314833931507]]
	 [0.0, 53.23529411764706]
